In [1]:
import pyspark

import os
from datetime import date
import functools
from IPython.core.display import display, HTML
#import findspark
#findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.sql.functions import count, isnan, mean, col, countDistinct, format_number, dayofmonth, hour, dayofyear, month, year, weekofyear, date_format, when,datediff, months_between

import re 
from functools import reduce

In [2]:
spark = SparkSession.builder.appName("vaccine_trans").getOrCreate()

# Read data

In [3]:
path = '/Users/julianeoliveira/Desktop/github/Datasets from the gitcomputations/Vaccine_covid/vacinacao_transformado.csv'

In [4]:
df = spark.read.csv(path, sep=',', header = True, encoding="utf-8", inferSchema = True)

In [23]:
df.limit(5).toPandas() 

,id,uf_res,mun_res,nome_mun_res,1ª dose,1ª dose revacinacao,1º reforco,2ª dose,2ª dose revacinacao,3ª dose,...,three,three_refor,undertermined,adicional,dose_inicial,revac,refor,uni,trat17,trat1
0,00002330ef1b9567153faa7edc1ab061f8d327033c034a...,mg,310710,boa esperanca,"2021-09-14,covid-19 pfizer - comirnaty",None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1,00003f27d1ba5af63189b1dc0a45801094c1c8ff15875f...,ba,292900,sao felix,"2021-07-06,covid-19 astrazeneca/fiocruz - covi...",None,None,"2021-09-28,covid-19 astrazeneca/fiocruz - covi...",None,None,...,0,0,0,0,0,0,0,0,0,0
2,00004d5067053ddb6683c90a07a65658e38d868a2763fc...,rj,330040,barra mansa,"2021-07-23,covid-19 sinovac/butantan - coronavac",None,None,"2021-08-25,covid-19 sinovac/butantan - coronavac",None,None,...,0,0,0,0,0,0,0,0,0,0
3,00005b66ccccbc8a83ab509f6e30314ede188280b4584e...,mt,510250,caceres,"2021-05-07,covid-19 astrazeneca/fiocruz - covi...",None,None,"2021-07-20,covid-19 astrazeneca/fiocruz - covi...",None,None,...,0,0,0,0,0,0,0,0,0,0
4,00006645047542a9c4396635a4c4ee99f7b5d1e643fb99...,rj,330270,marica,"2021-06-10,covid-19 astrazeneca/fiocruz - covi...",None,None,"2021-09-02,covid-19 astrazeneca/fiocruz - covi...",None,None,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df.columns

['id',
 'uf_res',
 'mun_res',
 'nome_mun_res',
 '1ª dose',
 '1ª dose revacinacao',
 '1º reforco',
 '2ª dose',
 '2ª dose revacinacao',
 '3ª dose',
 '3º reforco',
 'dose',
 'dose adicional',
 'dose inicial',
 'reforco',
 'revacinacao',
 'tratamento com dezessete doses',
 'tratamento com uma dose',
 'unica']

# Individuals with only one dose

In [12]:
df = df.withColumn("at_least_one",when(col('1ª dose').isNotNull(),1).otherwise(0))

In [8]:
df = df.withColumn("two",when(col('2ª dose').isNotNull(),1).otherwise(0))

In [9]:
df = df.withColumn("one_reva",when(col('1ª dose revacinacao').isNotNull(),1).otherwise(0))

In [10]:
df = df.withColumn("one_refor",when(col('1º reforco').isNotNull(),1).otherwise(0))

In [11]:
df = df.withColumn("two_reva",when(col('2ª dose revacinacao').isNotNull(),1).otherwise(0))

In [13]:
df = df.withColumn("three",when(col('3ª dose').isNotNull(),1).otherwise(0))

In [14]:
df = df.withColumn("three_refor",when(col('3º reforco').isNotNull(),1).otherwise(0))

In [15]:
df = df.withColumn("undertermined",when(col('dose').isNotNull(),1).otherwise(0))

In [16]:
df = df.withColumn("adicional",when(col('dose adicional').isNotNull(),1).otherwise(0))

In [17]:
df = df.withColumn("dose_inicial",when(col('dose inicial').isNotNull(),1).otherwise(0))

In [19]:
df = df.withColumn("refor",when(col('reforco').isNotNull(),1).otherwise(0))

In [18]:
df = df.withColumn("revac",when(col('revacinacao').isNotNull(),1).otherwise(0))

In [20]:
df = df.withColumn("uni",when(col('unica').isNotNull(),1).otherwise(0))

In [21]:
df = df.withColumn("trat17",when(col('tratamento com dezessete doses').isNotNull(),1).otherwise(0))

In [22]:
df = df.withColumn("trat1",when(col('tratamento com uma dose').isNotNull(),1).otherwise(0))

In [8]:
df.count()

163201565

In [9]:
163201565*100/213969727

76.27320335834237

In [10]:
163201565*100/193976530

84.13469660479028

# Count number of people vaccinated by state with or withour complete vaccine cicle

In [5]:
df.groupby('uf_res').count().coalesce(1).write.format("csv").save("/Users/julianeoliveira/Desktop/github/Datasets from the gitcomputations/Vaccine_covid/dose_uf.csv")#.show(truncate=False)

In [13]:
1502763 - 854580

648183

In [24]:
df.columns

['id',
 'uf_res',
 'mun_res',
 'nome_mun_res',
 '1ª dose',
 '1ª dose revacinacao',
 '1º reforco',
 '2ª dose',
 '2ª dose revacinacao',
 '3ª dose',
 '3º reforco',
 'dose',
 'dose adicional',
 'dose inicial',
 'reforco',
 'revacinacao',
 'tratamento com dezessete doses',
 'tratamento com uma dose',
 'unica',
 'only_one',
 'two',
 'one_reva',
 'one_refor',
 'two_reva',
 'at_least_one',
 'three',
 'three_refor',
 'undertermined',
 'adicional',
 'dose_inicial',
 'revac',
 'refor',
 'uni',
 'trat17',
 'trat1']

In [25]:
col_to_select = ['two',
 'one_reva',
 'one_refor',
 'two_reva',
 'at_least_one',
 'three',
 'three_refor',
 'undertermined',
 'adicional',
 'dose_inicial',
 'revac',
 'refor',
 'uni',
 'trat17',
 'trat1']

In [30]:
df.groupby(col_to_select).count().coalesce(1).write.format("csv").save("/Users/julianeoliveira/Desktop/github/Datasets from the gitcomputations/Vaccine_covid/dose_count_br.csv",header=True)#.show(truncate=False)